# Transformation toolbox

In [1]:
# TODO hide this cell ?

import mitsuba as mi

mi.set_variant("cuda_ad_rgb")

## Frame

The `mi.Frame3f` class stores a three-dimensional orthonormal coordinate frame. This class is very handy when you wish to convert vectors between different cartesian coordinates systems.

### Frame initialization

In [37]:
mi.Frame3f()  # Empty frame

mi.Frame3f(
    [1, 0, 0],  # s
    [0, 1, 0],  # t
    [0, 0, 1],  # n
)

mi.Frame3f([0, 1, 2])
# Makes use of `mi.coordinate_system` to compute the other two
# basis vectors

Frame[
  s = [[1, -0, -0]],
  t = [[-0, 0.666667, -1]],
  n = [[0, 1, 2]]
]

### Converting to/from local frames

- `mi.Frame3f.to_local`
- `mi.Frame3f.to_world`

Both of these methods are the main operations you will be using to convert between different coordinate frames.

In [38]:
frame = mi.Frame3f([1, 2, 3])

world_vector = mi.Vector3f([3, 2, 1])  # In world frame
local_vector = frame.to_local(world_vector)
local_vector

[[0.25, -3.5, 10.0]]

### Spherical coordinates

Mitsuba 3 provides convenience methods to efficiently compute certain trigonometric evaluations of spherical coordinates with respect to a `mi.Frame3f`. The naming convention used in these function is that theta is the elevation and phi is the azimuth.
For example, it provides `mi.Frame3f.sin_theta_2` and `mi.Frame3f.cos_phi`. The full list of these methods is availble in the [reference API() #TODO link.

## Transform

The `mi.Transform4f` and `mi.Transform3f` classes provides several functions to create common transformations, such as `translate`, `scale`, `rotate`, `look_at`. These are convenient to apply large transformations to a set of vertices for example. In fact, you can apply transforms to a `mi.Vector`, `mi.Point`, `mi.Normal` and even an `mi.Ray`. Note that all transforms are in homogenous coordiantes, `mi.Transform4f` can therefore be applied to 3-dimensional vectors and `mi.Transform3f` to 2-dimensional vectors.

Any `mi.Transform` object holds both the matrix corresponding the original transform and its inverse transpose. For convenience, there is also a `mi.Transform4f.inverse` method. All put together, this makes transforming back and forth straightforward.

#TODO Keep the non-Scalar/Scalar discussion here?
- `<type>`: type variant corresponding to the enabled backend (e.g. `Float, Vector3f, Transform4f`, ...)
- `Scalar<type>`: scalar variant type, agnostic to the enabled backend (e.g. `ScalarFloat, ScalarVector3f, ScalarTransform4f`, ...)

### Transform initialization


In [39]:
import numpy as np

# Default constructor is identity matrix
identity = mi.Transform4f()  

np_mat = np.array(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9],
    ]
)
mi_mat = mi.Matrix3f(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9],
    ]
)
list_mat = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]

# Build from different types
t_from_np = mi.Transform3f(np_mat)
t_from_mi = mi.Transform3f(mi_mat)
t_from_list = mi.Transform3f(list_mat)

# Broadcasting
t_from_value = mi.Transform3f(3)  # Scaled identity matrix
t_from_row = mi.Transform3f([3, 2, 3])  # Broadcast over matrix columns

### Translate

In [40]:
mi.Transform4f.translate([10, 20, 30])

[[[1, 0, 0, 10],
  [0, 1, 0, 20],
  [0, 0, 1, 30],
  [0, 0, 0, 1]]]

### Scale

In [41]:
mi.Transform4f.scale([10, 20, 30])

[[[10, 0, 0, 0],
  [0, 20, 0, 0],
  [0, 0, 30, 0],
  [0, 0, 0, 1]]]

### Rotate

In [42]:
mi.Transform4f.rotate(axis=[0, 1, 0], angle=10)

[[[0.984808, 0, 0.173648, 0],
  [0, 1, 0, 0],
  [-0.173648, 0, 0.984808, 0],
  [0, 0, 0, 1]]]

### Look at

In [43]:
mi.Transform4f.look_at(origin=[1, 0, 0], target=[0, 0, 0], up=[0, 0, 1])

[[[0, 0, -1, 1],
  [-1, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1]]]

### Orthohraphic/perspective

In [50]:
print(f"{mi.Transform4f.perspective(fov=45, near=0.1, far=10)=}")
print(f"{mi.Transform4f.orthographic(near=0.1, far=10)=}")

mi.Transform4f.perspective(fov=45, near=0.1, far=10)=[[[2.41421, 0, 0, 0],
  [0, 2.41421, 0, 0],
  [0, 0, 1.0101, -0.10101],
  [0, 0, 1, 0]]]
mi.Transform4f.orthographic(near=0.1, far=10)=[[[1, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 0.10101, -0.010101],
  [0, 0, 0, 1]]]


### From/to frame

⚠️ Only available for Transform4f

### Applying transforms

The Python `@` (`__matmul__`) operator can be used to apply `mi.Transform` objects to points, vectors, normals and rays or multiply transforms with other transforms. Depending on the operand's type, the operation has a slightly different output.

- `mi.Vector3f`: A typical matrix multiplication
- `mi.Point`: Adjusted matrix multiplication taking into account homogenous coordinates
- `mi.Normal`: Matrix multiplication using the inverse transpose to guarantee that the output is still normalized
- `mi.Ray`: Both the ray origin (`mi.Point`) and the ray direction (`mi.Vector`) are transformed with the `@` operator
- `mi.Transform`: 

#TODO Show chaining of operations too

In [63]:
t = mi.Transform4f.translate([0, 1, 2])
t = t * mi.Transform4f.scale([1, 2, 3])
v = mi.Vector3f([3, 4, 5])
p = mi.Point3f([3, 4, 5])
n = mi.Normal3f([3, 4, 5])


print(f"{t @ v=}")
print(f"{t @ p=}")
print(f"{t @ n=}")

t @ v=[[3.0, 8.0, 15.0]]
t @ p=[[3.0, 9.0, 17.0]]
t @ n=[[3.0, 2.0, 1.6666667461395264]]
